In [1]:
#fit notebook width
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [2]:
# MONGODB 
from pymongo import MongoClient

#Ploting
import folium
from folium.plugins import FastMarkerCluster
from folium.plugins import MarkerCluster
import matplotlib
import matplotlib.pyplot as plt
import ipywidgets as widgets

# Data and processing

import datetime
import numpy as np
import pandas as pd

# Printing
import pprint



In [3]:
client = MongoClient()
client.list_database_names()

['admin', 'config', 'local', 'osm', 'rennes', 'test_database']

In [4]:
# change rennes to another db name
db = client.rennes

In [5]:
db.list_collection_names()

['cars_data']

In [6]:
# change cars_data to another collection name
cars = db.cars_data


In [7]:
cars.count_documents({})

496410

In [8]:
df = pd.DataFrame(list(cars.find({})))

In [9]:
df.sort_values(by='time',inplace=True)

In [10]:
# this code create Trips 
def temporelTripsFilter(st,thresh= 15*60):
    # index of records where the difference between the two successive values is higher than thresh(default 15mins)
    return  np.where(True==np.array([(x-y).value/10**9 >thresh for x,y in zip(st['time'][1:],st['time'][:-1])]))[0]+1 

def stopsFilter(st,tempFilter,thresh= 10*60):
    sepIdx= np.empty(0, int)
    for segS,segE in zip(np.append(0,tempFilter),np.append(tempFilter,len(st['loc']))):
        seg=np.array(range(segS,segE))
        
        mainIdx=np.where([x!=0 for x in st['speed'][segS:segE]])[0]
    
        values=[st['time'][segS+i] for i in mainIdx]
        idx=np.where(True==np.array([(x-y).value/10**9 >thresh for x,y in zip(values[1:],values[:-1])]))[0]+1 
        
        np.append(sepIdx,[seg[j] for j in [mainIdx[i] for i  in idx]])
    shifts = np.array([(t['time'][i]-t['time'][i-1]).value/10**9 <thresh/2 for i in sepIdx],int)
    return np.append(tempFilter,sepIdx-shifts)

def segmentsOn(locs, idx):
    # segments
    return [[locs[cl][i:j] for cl in locs.index] for i,j in zip(np.append(0,idx),np.append(idx,len(locs['loc'])))]

def buildTrips(df):
    carsDF=df.groupby(by='id').agg(lambda locs: [ *locs ])
    trips =carsDF.apply(lambda x : list(zip(*segmentsOn(x,stopsFilter(x,temporelTripsFilter(x))))),axis=1)
    trips=pd.DataFrame([[trips.iloc[idx].name,*row] for idx in range(len(trips)) for row in zip(*trips.iloc[idx])],columns=['id',*trips.columns])
    trips=trips.assign(day=trips.time.apply(lambda x:x[0].date()))
    trips=trips.assign(begin=trips.apply(lambda x : dict([(c,x[c][0]) for c in ['loc','time','heading','speed','INSEE_iris_code']]),axis=1))
    trips=trips.assign(end=trips.apply(lambda x : dict([(c,x[c][len(x[c])-1]) for c in ['loc','time','heading','speed','INSEE_iris_code']]),axis=1))
    trips=trips.assign(dur=trips.time.apply(lambda x :(x[len(x)-1]-x[0]).value/((10**9)*60) ))

    return trips
    
trips= buildTrips(df)

In [11]:
folium_map = folium.Map(location=[48.14301,-1.69537],
                         zoom_start=13,
                         tiles="OpenStreetMap")
def DrawTrips():

    #FastMarkerCluster(data=dfGrouped.iloc[curID['loc'] ).add_to(folium_map)

    return folium_map

def addTrip(folium_map,trip,color='red'):
    # create lines and markers for the trip
    # MarkerCluster plots all points given to locations
    # PolyLine plots links between points given to location
    MarkerCluster(locations=[[x['coordinates'][1],x['coordinates'][0]] for x in trip['loc']],
                  icons = [folium.Icon(color='black', icon='play-circle'),*[folium.Icon(color=color, icon='info-sign')for _ in range(len(trip['loc'])-2)],folium.Icon(color='black', icon='stop') if len(trip['loc'])>1 else []],
                  popups=['ID : '+trip.id+'<br>Speed : '+str(s)+'<br>Time : '+ str(t) +'<br> cooRdinates : '+'lat : '+str(l['coordinates'][1])+' lon : '+str(l['coordinates'][0]) for s,t,l in zip(trip['speed'],trip['time'],trip['loc'])]).add_to(folium_map)
    folium.PolyLine(locations=[[x['coordinates'][1],x['coordinates'][0]] for x in trip['loc']],color=color).add_to(folium_map)

    #[folium.CircleMarker(location=x,radius=3).add_to(folium_map) for x in carsDF.iloc[curID]['loc']];

#     segments = segmentsOn(carsDF.iloc[curID],stopsFilter(carsDF.iloc[curID],temporelTripsFilter(carsDF.iloc[curID])))
#     colors=plt.cm.brg([ x/len(segments) for x in range(len(segments))])
#     for s,c  in zip(segments,colors):
#         folium.PolyLine(locations=s,color=matplotlib.colors.rgb2hex(c)).add_to(folium_map)


def addCarTrips(folium_map,trips,carsID):
    carTrips=trips[trips.id==carsID]
    colors=plt.cm.brg([ x/len(carTrips) for x in range(len(carTrips))])
    for s,c  in zip(carTrips.iterrows(),colors):
        addTrip(folium_map,s[1],matplotlib.colors.rgb2hex(c))
addTrip(folium_map,trips.iloc[0],'purple')
# addTrip(folium_map,trips,1,'blue')
# addTrip(folium_map,trips,2,'green')
# addTrip(folium_map,trips,3,'black')

# addCarTrips(folium_map,trips,'FFDF2DFA65A30E26D5C5CEC2FA2A8853')
# folium.TileLayer('Stamen Terrain').add_to(folium_map)
# folium.TileLayer('OpenStreetMap').add_to(folium_map)

DrawTrips()

In [12]:
# this part add interaction to the map 
maxV=10
tripIdWidget=widgets.IntSlider(min=0,max=maxV)
carIdWidget = widgets.Dropdown(options=list(trips.id.unique()[0:100]))
dateWidget=widgets.DatePicker(
    description='Pick a Date',
    disabled=False,
)

def printmap(carID=carIdWidget,byTrip=False,trip=tripIdWidget,byDate=False,date=dateWidget):
    carTrips=trips[trips.id==carID]    
    maxV= len(carTrips)-1
    if(byDate):
        carTrips=carTrips[carTrips.day ==date]
        maxV= len(carTrips)-1 if (len(carTrips)-1)>0 else 0
    folium_map = folium.Map(location=[48.10301,-1.65537],
                        zoom_start=13,
                        tiles="OpenStreetMap")
    if byTrip and len(carTrips)>0:
        if(trip>maxV):
            trip=0
        addTrip(folium_map,carTrips.iloc[trip],'purple')
        print('Start : ',carTrips.iloc[trip]['begin'],'\nEnd : ',carTrips.iloc[trip]['end'],'\nDur : '+str(carTrips.iloc[trip]['dur'])+' mins')
    else :
        addCarTrips(folium_map,carTrips,carID)
    display(folium_map)
    if(date == None or date>carTrips.day.max()):
        dateWidget.value=carTrips.day.max()
    if(date == None or date<carTrips.day.min()):
        dateWidget.value=carTrips.day.min()
    tripIdWidget.max=maxV
interact(printmap);


interactive(children=(Dropdown(description='carID', options=('0001D0DEE1DD5F61D5508ED7DC1F5303', '0005AE68C9CD…